# Introduction

Suppose you have never been to the US and you need to have just pizza while you are there. So you need to go to a spot with a high thickness of Pizza puts around you. The issue we plan to settle is to break down the Pizza stores' areas in the significant US urban communities and locate the best spot for our vacationer with the goal that he can have a decent pizz-ourism. Our primary objective are vacationers with a sample of western-style pizza

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
CLIENT_ID = 'ILXBHG35FKPOCDXT2ICHSAGAV2SRNJZPIZ4FAF4DEPR5E0LX' # your Foursquare ID
CLIENT_SECRET = 'E4M0PNQXU4HDRCMWTM5BDAOZDDCIENFW51AB5W3CJDJG3QTW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: ILXBHG35FKPOCDXT2ICHSAGAV2SRNJZPIZ4FAF4DEPR5E0LX


In [3]:
# type your answer here
LIMIT = 500 # Maximum is 100
cities = ["New York, NY", 'Chicago, IL', 'San Francisco, CA', 'Jersey City, NJ', 'Boston, MA']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1ca941735") # PIZZA PLACE CATEGORY ID
    results[city] = requests.get(url).json()

In [4]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [5]:
df_venues

{'New York, NY':                                    Name                      Address  \
 0                        Champion Pizza              17 Cleveland Pl   
 1                                B Side                370 W 51st St   
 2                   Prince Street Pizza                 27 Prince St   
 3                           Joe's Pizza                150 E 14th St   
 4                                 Marta                 29 E 29th St   
 5                                 Posto                  310 2nd Ave   
 6                           Joe's Pizza                1435 Broadway   
 7                           Joe's Pizza                 7 Carmine St   
 8                New York Pizza Suprema                  413 8th Ave   
 9                Vezzo Thin Crust Pizza            178 Lexington Ave   
 10                              Capizzi                  547 9th Ave   
 11        San Matteo Pizza Espresso Bar                 1739 2nd Ave   
 12            John's of Bleecker S

In [6]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of pizza places in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of pizza places in New York, NY =  302
Showing Top 100
Total number of pizza places in Chicago, IL =  226
Showing Top 100
Total number of pizza places in San Francisco, CA =  169
Showing Top 100
Total number of pizza places in Jersey City, NJ =  127
Showing Top 100
Total number of pizza places in Boston, MA =  184
Showing Top 100


In [7]:
maps[cities[0]]

In [8]:
maps[cities[1]]

In [9]:
maps[cities[2]]

In [10]:
maps[cities[3]]

In [11]:
maps[cities[4]]

We can see that New York and Jersey City are the most dense cities with Pizza places. And better than that, they are just one shore away.

Also, let's have a concrete measure of this density.

In [12]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New York, NY
Mean Distance from Mean coordinates
0.02264717448107267
Chicago, IL
Mean Distance from Mean coordinates
0.05863022035439393
San Francisco, CA
Mean Distance from Mean coordinates
0.028299772774669253
Jersey City, NJ
Mean Distance from Mean coordinates
0.029338830612325673
Boston, MA
Mean Distance from Mean coordinates
0.03506421050907873


In [13]:
maps[cities[0]]

In [14]:
maps[cities[1]]

In [15]:
maps[cities[2]]

In [16]:
maps[cities[3]]

In [17]:
maps[cities[4]]

We now see that New York is his best option, and its the Third best place is Jersey City which is just on the other side of the shore.